# Express Deep Learning in Python - Examples

We will run a couple of examples to see how different parameters affect the performance of the classifier.

In [31]:
import numpy
import keras
import os

from keras import backend as K
from keras import losses, optimizers, regularizers
from keras.datasets import mnist
from keras.layers import Activation, ActivityRegularization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.utils.np_utils import to_categorical

from keras.callbacks import TensorBoard

In [32]:
batch_size = 128
num_classes = 10
epochs = 10
TRAIN_EXAMPLES = 60000
TEST_EXAMPLES = 10000

# image dimensions
img_rows, img_cols = 28, 28

# load the data (already shuffled and splitted)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshape the data to add the "channels" dimension
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# normalize the input in the range [0, 1]
# to make quick runs, select a smaller set of images.
train_mask = numpy.random.choice(x_train.shape[0], TRAIN_EXAMPLES, replace=False)
x_train = x_train[train_mask, :].astype('float32')
y_train = y_train[train_mask]
test_mask = numpy.random.choice(x_test.shape[0], TEST_EXAMPLES, replace=False)
x_test = x_test[test_mask, :].astype('float32')
y_test = y_test[test_mask]

x_train /= 255
x_test /= 255

print('Train samples: %d' % x_train.shape[0])
print('Test samples: %d' % x_test.shape[0])

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Train samples: 60000
Test samples: 10000


### Convolutional 1

In [33]:
EXPERIMENT_COUNTER = 4

In [34]:
def write_summary(filename, model):
    with open(filename, 'w') as log_file:
        model.summary(print_fn=lambda x: log_file.write(x + '\n'))

In [48]:
def evaluate_model(model, experiment_name=EXPERIMENT_COUNTER):
    # train the model
    logs_dirname = './logs/experiment-{}'.format(experiment_name)
    tensorboard = TensorBoard(log_dir=logs_dirname, histogram_freq=0,
                          write_graph=False, write_images=False)
    epochs = 20
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test),
                        callbacks=[tensorboard])
    # TIP: write the model summary to keep track of your experiments
    write_summary(os.path.join(logs_dirname, 'model-summary.txt'), model)

    # evaluate the model
    return model.evaluate(x_test, y_test, verbose=0)

In [10]:
# define the network architecture
model = Sequential()
model.add(Conv2D(filters=16,
                 kernel_size=(3, 3),
                 strides=(1,1),
                 padding='valid',
                 activation='relu',
                 input_shape=input_shape,
                 activity_regularizer='l2'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# compile the model
model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'mae'])

In [14]:
evaluate_model(model)
EXPERIMENT_COUNTER += 1

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 37s - loss: 6.2937 - acc: 0.8517 - mean_absolute_error: 0.0595 - val_loss: 0.2786 - val_acc: 0.9516 - val_mean_absolute_error: 0.0195
Epoch 2/20
60000/60000 [==============================] - 36s - loss: 0.2862 - acc: 0.9376 - mean_absolute_error: 0.0229 - val_loss: 0.1703 - val_acc: 0.9662 - val_mean_absolute_error: 0.0129
Epoch 3/20
60000/60000 [==============================] - 36s - loss: 0.2088 - acc: 0.9526 - mean_absolute_error: 0.0165 - val_loss: 0.1470 - val_acc: 0.9701 - val_mean_absolute_error: 0.0095
Epoch 4/20
60000/60000 [==============================] - 36s - loss: 0.1781 - acc: 0.9600 - mean_absolute_error: 0.0139 - val_loss: 0.1284 - val_acc: 0.9719 - val_mean_absolute_error: 0.0090
Epoch 5/20
60000/60000 [==============================] - 36s - loss: 0.1593 - acc: 0.9628 - mean_absolute_error: 0.0124 - val_loss: 0.1049 - val_acc: 0.9771 - val_mean_absolute_erro

[0.04902356365174055, 0.98470000000000002, 0.0043716243533417579]

### Convolutional 2

In [46]:
# define the network architecture
model = Sequential()
model.add(Conv2D(filters=16,
                 kernel_size=(3, 3),
                 strides=(1,1),
                 padding='valid',
                 activation='sigmoid',
                 input_shape=input_shape,
                 activity_regularizer='l2'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

# compile the model
model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'mae'])

In [49]:
evaluate_model(model)
EXPERIMENT_COUNTER += 1

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 14s - loss: 2212.1909 - acc: 0.4919 - mean_absolute_error: 0.1331 - val_loss: 1478.8597 - val_acc: 0.8306 - val_mean_absolute_error: 0.0632
Epoch 2/20
60000/60000 [==============================] - 14s - loss: 1081.5625 - acc: 0.8475 - mean_absolute_error: 0.0536 - val_loss: 752.2486 - val_acc: 0.8850 - val_mean_absolute_error: 0.0405
Epoch 3/20
60000/60000 [==============================] - 14s - loss: 538.8397 - acc: 0.8803 - mean_absolute_error: 0.0415 - val_loss: 363.0824 - val_acc: 0.8976 - val_mean_absolute_error: 0.0368
Epoch 4/20
60000/60000 [==============================] - 14s - loss: 253.5612 - acc: 0.8886 - mean_absolute_error: 0.0391 - val_loss: 165.6639 - val_acc: 0.9091 - val_mean_absolute_error: 0.0329
Epoch 5/20
60000/60000 [==============================] - 14s - loss: 113.4703 - acc: 0.8901 - mean_absolute_error: 0.0391 - val_loss: 72.4159 - val_acc: 0.9119 - 